In [15]:
!pip install pandasql

  Preparing metadata (setup.py) ... done
  Created wheel for pandasql: filename=pandasql-0.7.3-py3-none-any.whl size=26771 sha256=de6c1948d3dab5fb7f8b7ac06c3dc2830570bb7233f11bd26c943c54e13bb8f3
  Stored in directory: /root/.cache/pip/wheels/e9/bc/3a/8434bdcccf5779e72894a9b24fecbdcaf97940607eaf4bcdf9
Successfully built pandasql


In [16]:
import pandas as pd
from pandasql import sqldf

In [3]:
!git clone https://github.com/kdaga81/IPL-Auction.git

Cloning into 'IPL-Auction'...


In [18]:
df = pd.read_csv("ipl_2022_dataset.csv", index_col=0)

df.head()

,Player,Base Price,TYPE,COST IN ₹ (CR.),Cost IN $ (000),2021 Squad,Team
0,Rashid Khan,Draft Pick,BOWLER,15.0,1950.0,SRH,Gujarat Titans
1,Hardik Pandya,Draft Pick,ALL-ROUNDER,15.0,1950.0,MI,Gujarat Titans
2,Lockie Ferguson,2 Cr,BOWLER,10.0,1300.0,KKR,Gujarat Titans
3,Rahul Tewatia,40 Lakh,ALL-ROUNDER,9.0,1170.0,RR,Gujarat Titans
4,Shubman Gill,Draft Pick,BATTER,8.0,1040.0,KKR,Gujarat Titans


In [19]:
print('Total number of Rows: ', df.shape[0])
print('Total number of Columns: ', df.shape[1])

Total number of Rows:  633
Total number of Columns:  7


In [20]:
df.dtypes

Player              object
Base Price          object
TYPE                object
COST IN ₹ (CR.)    float64
Cost IN $ (000)    float64
2021 Squad          object
Team                object
dtype: object

In [21]:
df['Base Price'].unique()

array(['Draft Pick', '2 Cr', '40 Lakh', '20 Lakh', '1 Cr', '75 Lakh',
       '50 Lakh', '30 Lakh', 'Retained', '1.5 Cr'], dtype=object)

In [22]:
df['Base Price'].value_counts()

Base Price
20 Lakh       344
50 Lakh       104
2 Cr           48
1 Cr           33
Retained       27
75 Lakh        26
1.5 Cr         20
40 Lakh        16
30 Lakh         9
Draft Pick      6
Name: count, dtype: int64

In [23]:
df['TYPE'].unique()

array(['BOWLER', 'ALL-ROUNDER', 'BATTER', 'WICKETKEEPER'], dtype=object)

In [24]:
df['TYPE'].value_counts(normalize=True)

TYPE
ALL-ROUNDER     0.382306
BOWLER          0.339652
BATTER          0.176935
WICKETKEEPER    0.101106
Name: proportion, dtype: float64

In [25]:
df2=df.rename(columns={'Player':'player',
                       'Base Price':'base_price',
                       'TYPE':'type',
                       'COST IN ₹ (CR.)':'cost_inr',
                       'Cost IN $ (000)': 'cost_usd',
                       '2021 Squad':'2021_team',
                       'Team':'2022_team'})

In [26]:
df2.head()

,player,base_price,type,cost_inr,cost_usd,2021_team,2022_team
0,Rashid Khan,Draft Pick,BOWLER,15.0,1950.0,SRH,Gujarat Titans
1,Hardik Pandya,Draft Pick,ALL-ROUNDER,15.0,1950.0,MI,Gujarat Titans
2,Lockie Ferguson,2 Cr,BOWLER,10.0,1300.0,KKR,Gujarat Titans
3,Rahul Tewatia,40 Lakh,ALL-ROUNDER,9.0,1170.0,RR,Gujarat Titans
4,Shubman Gill,Draft Pick,BATTER,8.0,1040.0,KKR,Gujarat Titans


In [27]:
df3 = df2.drop(['cost_usd'],axis=1)

In [28]:
df3.head()

,player,base_price,type,cost_inr,2021_team,2022_team
0,Rashid Khan,Draft Pick,BOWLER,15.0,SRH,Gujarat Titans
1,Hardik Pandya,Draft Pick,ALL-ROUNDER,15.0,MI,Gujarat Titans
2,Lockie Ferguson,2 Cr,BOWLER,10.0,KKR,Gujarat Titans
3,Rahul Tewatia,40 Lakh,ALL-ROUNDER,9.0,RR,Gujarat Titans
4,Shubman Gill,Draft Pick,BATTER,8.0,KKR,Gujarat Titans


In [29]:
mysql = lambda q: sqldf(q, globals())

In [30]:
mysql("""SELECT player, cost_inr FROM df3 WHERE type = 'BATTER' ORDER BY 2 DESC LIMIT 3""")

,player,cost_inr
0,Rohit Sharma,16.0
1,Virat Kohli,15.0
2,Kane Williamson,14.0


In [31]:
mysql("""SELECT player, cost_inr FROM df3 WHERE type = 'BOWLER' ORDER By 2 DESC LIMIT 5 """)

,player,cost_inr
0,Rashid Khan,15.00
1,Deepak Chahar,14.00
2,Jasprit Bumrah,12.00
3,Shardul Thakur,10.75
4,Lockie Ferguson,10.00


In [32]:
mysql(""" SELECT player, cost_inr FROM df3 WHERE type = 'WICKETKEEPER' AND cost_inr is not null ORDER BY 2 LIMIT 5 """)

,player,cost_inr
0,N. Jagadeesan,0.2
1,Baba Indrajith,0.2
2,Jitesh Sharma,0.2
3,Aryan Juyal,0.2
4,Luvnith Sisodia,0.2


In [33]:
mysql("""SELECT type, round(avg(cost_inr),2) average_price FROM df3 GROUP BY 1 ORDER BY 2 DESC""")

,type,average_price
0,WICKETKEEPER,5.09
1,BATTER,4.11
2,ALL-ROUNDER,3.61
3,BOWLER,3.07


In [34]:
mysql("""SELECT player, cost_inr FROM df3 WHERE base_price ="Retained" ORDER BY 2 DESC LIMIT 5 """)

,player,cost_inr
0,Ravindra Jadeja,16.0
1,Rishabh Pant,16.0
2,Rohit Sharma,16.0
3,Virat Kohli,15.0
4,Sanju Samson,14.0
